In [1]:
!pip install -q -r requirements.txt
!pip install -q ipywidgets
# GUI tools
%gui asyncio

# jupyter labextension install @jupyter-widgets/jupyterlab-manager


import asyncio
import ipywidgets as widgets
from IPython.display import display
from typing import AsyncIterable

# def wait_for_change(widget, value):
#     future = asyncio.Future()
#     def getvalue(change):
#         # make the new value available
#         future.set_result(change.new)
#         widget.unobserve(getvalue, value)
#     widget.observe(getvalue, value)
#     return future


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import chromadb
from chromadb.utils import embedding_functions

# You'll need to set replicate key as an environment env i.e. export REPLICATE_API_TOKEN=your_token
import replicate
# Tool for displaying steamed text output

In [6]:
async def get_replicate_stream(user_input: str) -> AsyncIterable[str | bytes]:
    try:
        # The replicate/llama-2-70b-chat model can stream output as it's running.
        # The predict method returns an iterator, and you can iterate over that output.
        # Prompting guide https://replicate.com/blog/how-to-prompt-llama
        output: Iterator = replicate.run(
            "replicate/llama-2-70b-chat:2c1608e18606fad2812020dc541930f2d0495ce32eee50074220b87300bc16e1",
            input={
                "prompt": f"[INST] {user_input} [/INST]",
                "system_prompt": "",
            },
        )
        for item in output:
            # Dumb as hell - streaming only works if you introduce a tiny delay
            # Hypothesis
            # The delay essentially serves as a workaround by allowing the client to catch up and request the
            # next chunk of data. It's worth investigating further if there are other underlying issues,
            # such as buffering in your server or any intermediaries, that might affect real-time streaming.
            await asyncio.sleep(0.000001)  # Introducing a delay
            yield item
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

In [11]:
from functools import wraps
from ipywidgets import IntSlider, VBox, HTML, Layout

def yield_for_change(widget, attribute):
    """Pause a generator to wait for a widget change event.
        
    This is a decorator for a generator function which pauses the generator on yield
    until the given widget attribute changes. The new value of the attribute is
    sent to the generator and is the value of the yield.
    """
    def f(iterator):
        @wraps(iterator)
        def inner():
            i = iterator()
            def next_i(change):
                try:
                    i.send(change.new)
                except StopIteration as e:
                    widget.unobserve(next_i, attribute)
            widget.observe(next_i, attribute)
            # start the generator
            next(i)
        return inner
    return f

# Button click event handler
async def on_button_click(b):
    output_area.value = ""  # Clear the existing output
    async for item in get_replicate_stream(input_text.value):
        output_area.value += item  # Add new output lines to the Textarea

# Create ipywidgets
input_text = widgets.Text(
    value='',
    placeholder='Type your input',
    description='Input:',
    disabled=False
)

output_area = widgets.Textarea(
    value='',
    placeholder='Output will be displayed here',
    description='Output:',
    disabled=False,
    layout=Layout(width='90%', height='200px')
)

run_button = widgets.Button(
    description='Run',
    disabled=False,
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run',
    icon='play'  # (FontAwesome names without the `fa-` prefix)
)

# Attach event handler
run_button.on_click(lambda b: asyncio.create_task(on_button_click(b)))

# Display widgets
display(input_text, run_button, output_area)

Text(value='', description='Input:', placeholder='Type your input')

Button(description='Run', icon='play', style=ButtonStyle(), tooltip='Run')

Textarea(value='', description='Output:', layout=Layout(height='200px', width='90%'), placeholder='Output will…